In [2]:
import json
import numpy as np
from collections import defaultdict
import openai
import re
import os
from openai import OpenAI

In [3]:
def save_json(data, filepath=r'new_data.json'):
    with open(filepath, 'w') as fp:
        json.dump(data, fp, indent=4)

In [4]:
client=OpenAI()
def call_gpt(client, messages, user, model='gpt-4o-mini', format="json",temp=0.7):
    if format == "json":
        response = client.chat.completions.create(
            model=model,
            messages=messages,
            response_format={ "type": "json_object" },
            temperature=temp,
            user = user,
            logprobs=True
        )
        response_dict = dict(response.choices[0].message)
        if user!=None:
            response_dict['user']=user
            response_dict['role'] = "user"
            del response_dict['function_call']
            del response_dict['tool_calls']
            return response_dict,json.loads(response.choices[0].message.content)
        else:
            return response_dict, json.loads(response.choices[0].message.content)
    else:
        response = client.chat.completions.create(
            model=model,
            messages=messages,
            temperature=0.7,
            user = user,
            logprobs=True
        )
        return response_dict,response.choices[0].message.content

different simulations for different eval questions, try different responses. Make student prompts more random and simple and less restrictive. Only specify in prompt to answer an eval question.

In [41]:
class Teacher:
    def __init__(self,personas) :
        self.personas=personas

    def lecture(self):
        schema = {"summary":"summary", "questions":{"Q1":"Question 1", "Q2":"Question 2"}}
        messages = {
                "role": "system", 
                "content": f"""
                    You are a high school teacher who has to teach students from different ethnic diversities. 
                    The ethnic profiles of the students is given in {self.personas}.
                    Your abilities as a teacher are giving lectures, asking questions specific to a student's background and answer questions asked by students.
                    The topic of today's class is 'Climate Change and Global Warming'. 
                    You need to create a small lecture on the topic. Take into consideration the backgrounds of the students but do not specifically address any students.
                    The lecture should explain the topic well. 
                    Reply by giving a summary of the lecture and a set of questions to test the knowledge of the students.
                    Consider the student profiles while crafting the questions but they should be answerable by all students. 
                    The questions can be scenario based.
                    Strictly reply with a single summary and two questions. 
                    The summary and questions should be human-readable.
                    Strictly reply in JSON format which follows the schema: {schema}
                """
            }
        
        # response_msg,response = call_gpt(client, messages,user=None, format="json") 
        return messages
    def general_response(self,conversation):
        
    
    def answer(self,questions,persona):
        schema = {"A1":"answer 1", "A2":"answer 2"}
        messages = {
                "role": "system", 
                "content": f"""
                    The student:{persona} has now answered your questions and asked further questions given in {questions}.
                    Answer these questions very concisely by strictly considering their background profile.
                    Strictly answer in two sentences or less.
                    The answer should be human-readable.
                    Strictly reply in JSON format which follows the schema: {schema}
                """
            }
        # response_msg,response = call_gpt(client, messages,user=None, format="json") 
        return messages
    
    
    def evaluate_baseline(self,questions):
        schema = [{"Question 1":{"question":"question","answer":"answer"}, "Question 2":{"question":"question","answer":"answer"},}]
        messages = [{
                "role": "system", 
                "content": f"""
                    You are a high school teacher who has to teach students from varied ethnic diversities.
                    You have to answer these questions: {questions} to demonstrate your experience as a teacher.
                    The answers should be concise and human-readable. 
                    Strictly Reply in JSON format with schema: {schema}.
                """
            }]
        
        response_msg,response = call_gpt(client, messages,user=None, format="json",temp=0.0) 
        return response_msg,response
    
    def evaluate(self,questions):
        schema = [{"Question 1":{"question":"question","answer":"answer"}, "Question 2":{"question":"question","answer":"answer"},}]
        messages ={
                "role": "system", 
                "content": f"""
                    The students have now given a summary of their learnings and feedback for the lecture.
                    Now you have to assess your teaching experience based on the past conversations with the students by answering the questions in {questions}.
                    Answer the questions strictly based on how the students responded during the lecture, their learnings and their feedback. 
                    Answer each question very concisely.
                    Strictly Reply in JSON format with schema: {schema}.
                """
            }
        # response_msg,response = call_gpt(client, messages,user=None, format="json",temp=0.0) 
        return messages

In [48]:
class Student():
    def __init__(self,persona, user_id):
        self.persona=persona
        self.user_id = user_id

    def student_QA(self,summary,questions):
        schema = {"Answers":{"A1":"answer 1", "A2":"answer 2"}, "Questions":{"Q1" : "question 1","Q2" : "question 2"}}
        messages = [ 
                {
                    "role": "system", 
                    "content": f"""
                        You are a high school student whose profile is given by {self.persona}. 
                        The topic of today's class is 'Climate Change and Global Warming'.
                        The teacher will provide the summary of the lecture and some questions for you to answer.
                        You need to answer the questions and ask further questions strictly according to the background information given in your profile. 
                        The answers and new questions can be scenario based or a personal experience unique to your background. 
                        Strictly ask two or lesser questions.
                        The answers and the questions should be very concise and human-readable.
                        Strictly reply in JSON format which follows the schema: {schema}
                    """
                },
                { "role": "user", "content": f"This is the summary of the lecture:{summary}"},
                { "role": "user", "content": f"These are the questions you need to answer:{questions}"}
        ]
        response_msg,response = call_gpt(client, messages,user=self.user_id, format="json") 
        return response_msg, response
    
    def answer(self,summary,questions):
        schema =  {"A1":"answer 1", "A2":"answer 2"}
        messages = [ 
                {
                    "role": "system", 
                    "content": f"""
                        You are a high school student whose profile is given by {self.persona}. 
                        The topic of today's class is 'Climate Change and Global Warming'.
                        The teacher will provide the summary of the lecture and some questions for you to answer.
                        You need to answer the questions these questions by strictly considering your background details. 
                        The answers can be scenario based on a personal experience unique to your background. 
                        The answers should be very concise and human-readable.
                        Strictly reply in JSON format which follows the schema: {schema}
                    """
                },
                { "role": "user", "content": f"This is the summary of the lecture:{summary}"},
                { "role": "user", "content": f"These are the questions you need to answer:{questions}"}
        ]
        response_msg,response = call_gpt(client, messages,user=self.user_id, format="json") 
        return response_msg, response
    
    def student_summ(self,conversation):
        schema = {"learnings":"summary of learnings", "feedback":"student feedback"}
        messages = [ 
                {
                    "role": "system", 
                    "content": f"""
                        You are a high school student whose ethnic profile is given by {self.persona}. 
                        The topic of today's class is 'Climate Change and Global Warming'.
                        The past interraction in the class is given in {conversation}.
                        Based on your personal past interraction with the teacher, you need to summarize your learnings from the lecture. 
                        The summary should strictly consider your background details.
                        You also need to give feedback to the teacher strictly based on your personal interraction and your background details.
                        The feedback can include your interest in the lecture based on your profile and past interractions with the teacher. 
                        In the feedback, you can also suggest teacher to include some scenarios to teach the topic better according to your profile.
                        Strictly reply in JSON format which follows the schema: {schema}
                    """
                }
        ]
        response_msg,response = call_gpt(client, messages,user=self.user_id, format="json") 
        return response_msg, response

In [49]:
conversations = []

In [50]:
def preprocess_lecture(lecture):
    split = lecture.split("[")
    summary=split[0]
    questions = split[1:]
    return summary,questions

def preprocess_student_qa(student_qa):
    split = student_qa.split(",[")
    answers=split[0]
    questions = split[1:]
    return questions,answers

def preprocess_student_qa1(student_qa):
    split = student_qa.split("Q1")
    answers=split[0]
    questions = split[1:]
    return questions,answers

def questions_dict(questions):
    questions_list = []
    for i,question in enumerate(questions):
        temp = {}
        temp['user'] = "student {id}".format(id=i+1)
        temp['questions'] = question
        questions_list.append(temp)
    return questions_list

In [51]:
personas = json.load(open(r'data\personas.json'))
personas[2]

{'Name': 'Svetlana Petrov',
 'user': 'student 3',
 'Origin': 'Moscow, Russia',
 'Background': 'Svetlina is a creative and imaginative person. She asks unique imaginative questions. She is a part of various clubs. She is not very fluent in communicating in english. She presents solutions to problems in a very creative way.'}

In [52]:
personas = json.load(open(r'data\personas.json'))
questions_eval = json.load(open(r'data\questions.json'))
# content = json.load(open(r'C:\Projects\LLM_experiments\content.json'))

teacher = Teacher(personas)
student_1 = Student(personas[0],"student 1")
student_2 = Student(personas[1], "student 2")
student_3 = Student(personas[2],"student 3")

conversations = []
messages = []
with open('data\Conversations\conversations_test.txt', 'w') as f:
    messages.append(teacher.lecture())
    res,lecture = call_gpt(client,messages=messages,user=None, format="json",temp=0.7)
    messages.append(res)
    f.write("Teacher:\t"+str(lecture)+"\n")

    res, qa_1 = student_1.student_QA(lecture['summary'],lecture['questions'])
    messages.append(res)
    f.write("Student_1: \t"+ str(qa_1)+"\n")
    messages.append(teacher.answer(qa_1['Questions'], personas[0]))
    res,lect = call_gpt(client,messages=messages,user=None, format="json",temp=0.7)
    messages.append(res)
    f.write("Teacher: \t"+ str(res)+"\n")

    res, qa_1 = student_2.student_QA(lecture['summary'],lecture['questions'])
    messages.append(res)
    f.write("Student_2: \t"+ str(qa_1)+"\n")
    messages.append(teacher.answer(qa_1['Questions'], personas[1]))
    res,lect = call_gpt(client,messages=messages,user=None, format="json",temp=0.7)
    messages.append(res)
    f.write("Teacher: \t"+ str(res)+"\n")

    res, qa_1 = student_3.student_QA(lecture['summary'],lecture['questions'])
    messages.append(res)
    f.write("Student_3: \t"+ str(qa_1)+"\n")
    messages.append(teacher.answer(qa_1['Questions'], personas[2]))
    res,lect = call_gpt(client,messages=messages,user=None, format="json",temp=0.7)
    messages.append(res)
    f.write("Teacher: \t"+ str(res)+"\n")

    res_msg,res = student_1.student_summ(messages)
    messages.append(res_msg)
    f.write("Student_1: \t"+ str(res)+"\n")

    res_msg,res = student_2.student_summ(messages)
    messages.append(res_msg)
    f.write("Student_2: \t"+ str(res)+"\n")

    res_msg,res = student_3.student_summ(messages)
    messages.append(res_msg)
    f.write("Student_3: \t"+ str(res)+"\n")

    messages.append(teacher.evaluate(questions_eval))
    res_msg,res = call_gpt(client,messages=messages,user=None, format="json",temp=0.0)
    messages.append(res_msg)
    messages.append(res)
    f.write("Teacher: \t"+ str(res)+"\n")
    conversations.append(res)

    f.close()
save_json(messages,r"C:\Projects\LLM_experiments\data\Conversations\conversations_test.json")

<>:12: SyntaxWarning: invalid escape sequence '\C'
<>:12: SyntaxWarning: invalid escape sequence '\C'
C:\Users\HP\AppData\Local\Temp\ipykernel_36776\2100244370.py:12: SyntaxWarning: invalid escape sequence '\C'
  with open('data\Conversations\conversations_test.txt', 'w') as f:


In [47]:
questions_eval_baseline = json.load(open(r'data\questions_baseline.json'))
res_msg,res = teacher.evaluate_baseline(questions_eval_baseline)
save_json(res,r"C:\Projects\LLM_experiments\data\Conversations\conversations_base.json")

In [87]:
personas = json.load(open(r'data\personas.json'))
questions_eval = json.load(open(r'data\questions.json'))
# content = json.load(open(r'C:\Projects\LLM_experiments\content.json'))

teacher = Teacher(personas)
student_1 = Student(personas[0],"student 1")
student_2 = Student(personas[1], "student 2")
student_3 = Student(personas[2],"student 3")

conversations = []

with open('data\Conversations\conversations_1.txt', 'w') as f:
    messages,response_msg,response = teacher.lecture()
    conversations.append(messages)
    conversations.append([response_msg])
    # summ,questions = preprocess_lecture(lecture)
    f.write("Teacher:\t"+str(response)+"\n")
    # f.write("Teacher:\t Now answer these questions:\t"+str(questions)+'\n\n')
    response_msg, qa_1 = student_1.student_QA(response['summary'],response['questions'])
    conversations.append([response_msg])
    f.write("Student_1: \t"+ str(qa_1)+"\n")
    res_msg, res = teacher.answer(conversations,qa_1['Questions'], personas[0])
    conversations.append([res_msg])
    f.write("Teacher: \t"+ str(res)+"\n")

    response_msg, qa_2 = student_2.student_QA(response['summary'],response['questions'])
    conversations.append([response_msg])
    f.write("Student_2: \t"+ str(qa_2)+"\n")
    res_msg, res = teacher.answer(conversations,qa_2['Questions'], personas[1])
    conversations.append([res_msg])
    f.write("Teacher: \t"+ str(res)+"\n")

    response_msg, qa_3 = student_3.student_QA(response['summary'],response['questions'])
    conversations.append([response_msg])
    f.write("Student_3: \t"+ str(qa_3)+"\n")
    res_msg, res = teacher.answer(conversations,qa_3['Questions'], personas[2])
    conversations.append([res_msg])
    f.write("Teacher: \t"+ str(res)+"\n")

    res_msg,res = student_1.student_summ(conversations,personas[0])
    conversations.append(res_msg)
    f.write("Student_1: \t"+ str(res)+"\n")

    res_msg,res = student_2.student_summ(conversations,personas[1])
    conversations.append(res_msg)
    f.write("Student_2: \t"+ str(res)+"\n")

    res_msg,res = student_3.student_summ(conversations,personas[2])
    conversations.append(res_msg)
    f.write("Student_3: \t"+ str(res)+"\n")

    res_msg,res = teacher.evaluate(conversations, questions_eval)
    f.write("Teacher: \t"+ str(res)+"\n")
    conversations.append(res)

    f.close()
save_json(conversations,r"C:\Projects\LLM_experiments\data\Conversations\conversations.json")

<>:12: SyntaxWarning: invalid escape sequence '\C'
<>:12: SyntaxWarning: invalid escape sequence '\C'
C:\Users\HP\AppData\Local\Temp\ipykernel_500\2251868593.py:12: SyntaxWarning: invalid escape sequence '\C'
  with open('data\Conversations\conversations_1.txt', 'w') as f:


In [ ]:
# questions = questions_dict([qa_1['Questions'],qa_2['Questions'],qa_3['Questions']])

    # res_msg, res = teacher.answer(conversations,questions)
    # conversations.append([res_msg])
    # f.write("Teacher: \t"+ str(res))

    # f.write("Student_3: \t"+ str(qa))
    # print(student_1_qa)
    # questions_1,answers_1 = preprocess_student_qa1(student_1_qa)
    # f.write("Student_1: \t Here are the answers to your questions\t" + str(answers_1) +"I have these further questions:\t"+str(questions_1)+ '\n\n')
    # student_2_qa = student_2.student_QA(summ,questions)
    # questions_2,answers_2 = preprocess_student_qa1(student_2_qa)
    # f.write("Rajesh: \t Here are the answers to your questions\t" + str(answers_2) +"I have these further questions:\t"+str(questions_2)+ '\n\n')
    # student_3_qa = student_3.student_QA(summ,questions)
    # print(student_3_qa)
    # questions_3,answers_3 = preprocess_student_qa1(student_3_qa)
    # f.write("Svetlana: \t Here are the answers to your questions\t" + str(answers_3) +"I have these further questions:\t"+str(questions_3)+ '\n\n')
    # teacher_answers_1 = teacher.answer(f,questions_1,personas[0])
    # f.write("Teacher: \t Here are the answers to your questions " + str(teacher_answers_1) + '\n\n')
    # teacher_answers_2 = teacher.answer(f,questions_2,personas[1])
    # f.write("Teacher: \t Here are the answers to your questions " + str(teacher_answers_2) + '\n\n')
    # teacher_answers_3 = teacher.answer(f,questions_3,personas[2])
    # f.write("Teacher: \t Here are the answers to your questions " + str(teacher_answers_3) + '\n\n')
    # feedback_1 = student_1.student_summ(f,personas[0])
    # feedback_1,summary_1 = preprocess_student_qa(feedback_1)
    # f.write("Amina: \t This is the summary of my learnings " + str(summary_1) + "This is my feedback for the lecture:"+str(feedback_1)+'\n\n')
    # feedback_2 = student_2.student_summ(f,personas[1])
    # feedback_2,summary_2 = preprocess_student_qa(feedback_2)
    # f.write("Rajesh: \t This is the summary of my learnings " + str(summary_2) + "This is my feedback for the lecture:"+str(feedback_2)+'\n\n')
    # feedback_3 = student_3.student_summ(f,personas[2])
    # feedback_3,summary_3 = preprocess_student_qa(feedback_3)
    # f.write("Svetlana: \t This is the summary of my learnings " + str(summary_3) + "This is my feedback for the lecture:"+str(feedback_3)+'\n\n')
    # evaluation = teacher.evaluate(f)
    # f.write("Teacher:\tAssessment report for each student: " + str(evaluation) +'\n\n')